# Decision theory project

In [89]:
import numpy as np
import pulp
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import pdist, squareform

##  Tasks 1:  Determination of the value function 

### a. The DM gives his preferences by comparing each pair of alternatives following the Macbeth questioning procedure.

![Image](./matrice_DM.png)

* The DM has given his preferences in this matrix with 6 types of preferences : extrem, very strong, strong, moderate, weak, very weak

### b. The analyst checks with the DM possible (inconsistencies) in the preferences especially transitivity

* The software MacBeth has done the work : inconstenciy especially transitivity is satisfied

### c. If the binary relation representing the preferences is a weak order, then a value function exists to represent these preferences.


* The binary relation is a weak order so a value function exists.

### d. Assess the value function (solving an optimization problem).

* First, the matrix of the DM is : (with 0:"Nulle",1:"Very Weak",2:"Weak",3:"Moderate",4:"Strong",5:"Very Strong" & -1 because it is symetric)

In [73]:
scale = {
    0: "Nulle",
    1: "Very Weak",
    2: "Weak",
    3: "Moderate",
    4: "Strong",
    5: "Very Strong"
}

DM_matrix=np.array([[0,3,4,4,4,6,6],
                    [-1,0,3,4,4,5,5],
                    [-1,-1,0,4,4,5,5],
                    [-1,-1,-1,0,3,4,4],
                    [-1,-1,-1,-1,0,4,4],
                    [-1,-1,-1,-1,-1,0,2],
                    [-1,-1,-1,-1,-1,-1,0]])

In [74]:
'''
Define a function which resolves the optimization problem from the DM_matrix
'''
def find_value_function(DM_matrix):
    #we use the pulp solver
    optimization_problem = pulp.LpProblem("Value_Function", pulp.LpMinimize) #solving an optimization problem
    values = pulp.LpVariable.dicts("Value", range(len(DM_matrix)), lowBound=0)
    optimization_problem += pulp.lpSum([values[i] for i in range(len(DM_matrix))]) 

    for i in range(len(DM_matrix)):
        for j in range(len(DM_matrix[0])):
            if DM_matrix[i][j] != -1: # to only treat the upper diagonal
                optimization_problem += values[i] >= DM_matrix[i][j] + values[j]

    #solve the probleme and recup the value of each class
    optimization_problem.solve()
    value_function = [pulp.value(values[i]) for i in range(len(DM_matrix))]

    return value_function

* Application on our matrix of social networks

In [75]:
value_function = find_value_function(DM_matrix)
social_networks = ["Sup", "Instagram", "LinkedIn", "X", "Facebook", "Tik Tok", "Inf"]

print("Our final value function :")
print(social_networks)
print(value_function)


Our final value function :
['Sup', 'Instagram', 'LinkedIn', 'X', 'Facebook', 'Tik Tok', 'Inf']
[19.0, 16.0, 13.0, 9.0, 6.0, 2.0, 0.0]


### e. A ranking can be established regarding the value obtained by each Social network.

Thanks to the value function, the ranking is :
* 1. Instagram
* 2. LinkedIn
* 3. X 
* 4. Facebook
* 5. Tik Tok

## Tasks 2: Supervised classification using expected utility (application to plastics sorting problem)

### a. Identify the utility function.

In [76]:
#From internet data, the utility matrix is
utility_matrice = [[1,0.25,0.5,0.75],[0.75,1,0.25,0.5],[0.5,0.75,1,0.25],[0.25,0.5,0.75,1]]

### b. Implement the expected utility model.

In [77]:
def EU(predicted, Pabs, Phips, Ppe, Ppp):
    #print('predicted : ',predicted)
    if predicted == 'ABS':
        return (1 * Pabs + 0.25 * Phips + 0.5 * Ppe + 0.75 * Ppp)
    elif predicted == 'HiPS':
        return (0.75 * Pabs + 1 * Phips + 0.25 * Ppe + 5 * Ppp)
    elif predicted == 'PP':
        return (0.5 * Pabs + 0.75 * Phips + 1 * Ppe + 0.25 * Ppp)
    elif predicted == 'PE':
        return (0.25 * Pabs + 0.5 * Phips + 0.75 * Ppe + 1 * Ppp)
    else:
        return None

In [78]:
'''
    Choice returns the class of the plastic using the expected utility
'''

def Choice(Data_to_predict, model):
    a = model.predict_proba(Data_to_predict)  # Use predict_proba to get probabilities
    act1 = EU('ABS', a[0][0], a[0][1], a[0][2], a[0][3])
    act2 = EU('HiPS', a[0][0], a[0][1], a[0][2], a[0][3])
    act3 = EU('PE', a[0][0], a[0][1], a[0][2], a[0][3])
    act4 = EU('PP', a[0][0], a[0][1], a[0][2], a[0][3])
    v = [act1, act2, act3, act4]

    max_index = v.index(max(v))
    plastic = ['ABS', 'HiPS', 'PE', 'PP']
    return plastic[max_index]

### c. Test your algorithm on the validation data of plastics (split your data into train and validation).


* Importation of data

In [79]:
X = pd.read_csv('./PlasticsTrain.csv',sep=';')

X.drop(['line', 'column', 'object'], axis=1, inplace=True)

# target column
Y = X['class']

# whole dataset
big_df = X

# X columns
X.drop('class',axis = 1, inplace=True)

#replace , to .
X = X[X.columns].apply(lambda x: x.str.replace(',', '.')).astype(float)

X.head()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [80]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)

score = rf_model.score(X_test,y_test)
print('Score without expected utility : ',score)

Score without expected utility :  0.9107752315943443


* Get the accuracy on the test set

In [81]:
def EU_algo(model):    
    accuracy_score = 0
    for i in range(len(X_test)-1):
        to_test = X_test.iloc[i:i+1]
        predicted = Choice(to_test,model)
        if predicted == y_test.iloc[i]:
            accuracy_score+=1
    return accuracy_score

accuracy_eu_rf = EU_algo(rf_model)
print('Score with expected utility with random forest : ', accuracy_eu_rf/len(y_test))

Score with expected utility with random forest :  0.36470014627011216


### d. Compare the performances of your algorithm using several classical prediction algorithms on the test plastic data (hidden at the moment).

* Using 2 other models

In [83]:
svm_model = SVC(probability=True)
knn_model = KNeighborsClassifier()

svm_model.fit(X_train, y_train)
knn_model.fit(X_train, y_train)

accuracy_eu_svm = EU_algo(svm_model)
accuracy_eu_knn = EU_algo(knn_model)

print('Score with expected utility with SVM : ', accuracy_eu_svm/len(y_test))
print('Score with expected utility with KNN : ', accuracy_eu_knn/len(y_test))

Score with expected utility with SVM :  0.3778644563627499
Score with expected utility with KNN :  0.4129692832764505


* The knn classifier seems to give the best results

## Tasks 3: Supervised classification using evidential KNN (application to plastics sorting problem)

### a. Identify the utility function.

For this question, we keep the same utility function than the Task 2.

In [92]:
import pandas as pd
import numpy as np

# u = [[1,0.6,0,0,0],
#      [0.6,1,0,0],
#      [0.5,0.5,0.8,0.4],
#      [0.5,0.5,0.4,0.8]]

# utility_matrix = pd.DataFrame(u, columns=classe)

# list of the classes
classe = ['ABS', 'HiPS', 'PE', 'PP', 'THETA']

In [2]:
# utility_matrix.head()

### b. Implement eknn.


* The train and test datasets are the same than Task 2.

Then, the dist_matrix is created : it permits to know all the Euclidean distances between X_i and X_j

In [88]:
# Distances
newDF = pd.concat([X_train, X_test])

# Convertir toutes les colonnes en float
newDF = newDF.replace(',', '.', regex=True)
newDF = newDF.astype(float)

distances = pdist(newDF.values, metric='euclidean')
dist_matrix = squareform(distances)

In [90]:
'''
    fit_gamma permits to get the list of the coefficients [gamma_1, ..., gamma_n]
    coefficients are fitted with the training dataset
'''
def fit_gamma(train_label, dist, classes):
    gamma = []
    nbC = len(classes)
    for i in range(nbC-1):
        C_indices = [t for t in range(len(train_label)) if (train_label[t] == classes[i])]
        C_dist = []
        for ci in range(len(C_indices)):
            for cj in range(ci + 1, len(C_indices)):
                C_dist.append(dist[C_indices[ci]][C_indices[cj]])
        gamma.append(1 / np.mean(C_dist))
    return gamma

In [93]:
# Observing the result
coeff_gamma = fit_gamma(y_train.values, dist_matrix, classe)
print(coeff_gamma)

[0.3822991405629782, 0.3723211618324772, 0.2142873527831586, 0.19209598569774933]


The parameter phi = [phi_1, ..., phi_n] is calculated thanks to the choice : phi_i(d) = alpha exp(-gamma *d ^ 2).

In [94]:
#we fix alpha = 0.95
def phi(gamma, d, alpha=0.95):
    return [alpha*np.exp(-gamma[i]*d**2) for i in range(len(gamma))]

In [95]:
''' 
    voisins is a function which returns the index of the k nearest neighbours from train dataset of a vector X. 
'''

def voisins(index, train_size, y, k):
    index_sort = np.argsort(dist_matrix[index])
    indice_voisins = []

    iteration = 1
    while len(indice_voisins) < k:
        # only index from the training set
        if(index_sort[iteration]<train_size):
            indice_voisins.append(index_sort[iteration])    
        iteration += 1

    distance_voisins = dist_matrix[index, indice_voisins]
    classes = y[indice_voisins]
    
    return indice_voisins, distance_voisins, classes

In [105]:
'''
    m_j compute the piece of evidence and combine the mass functions mj(.|x) using Dempster s rule
    It returns a list associated with each class
'''

def m_j(index, train_size, y, k):
    #collect the information of the neighbours
    indice_voisins, distance_voisins, classes = voisins(index, train_size, y, k)

    # init coeff_ABS 
    coeff_ABS, coeff_HIPS, coeff_PE, coeff_PP = 1, 1, 1, 1

    # compute the numerator of the coefficients of each classes
    for i in range(len(indice_voisins)) :
        current_class = str(classes.iloc[i])
        if current_class == 'ABS':
            coeff_ABS *= (1- phi(coeff_gamma, distance_voisins[i])[0])
        elif current_class == 'HiPS':
            coeff_HIPS *= (1- phi(coeff_gamma, distance_voisins[i])[1])
        elif current_class == 'PE':
            coeff_PE *= (1- phi(coeff_gamma, distance_voisins[i])[2])
        elif current_class == 'PP':
            coeff_PP *= (1- phi(coeff_gamma, distance_voisins[i])[3])
    
    # calcul the fourth m_i(A|X) = [A=theta_i, A!=theta_i]
    m_ABS=[1-coeff_ABS, coeff_ABS]
    m_HIPS=[1-coeff_HIPS, coeff_HIPS]
    m_PE=[1-coeff_PE, coeff_PE]
    m_PP=[1-coeff_PP, coeff_PP]

    m_j =  np.array([m_ABS, m_HIPS, m_PE, m_PP])

    # normalization coeff to compute m 
    product = np.prod(m_j) # empty prediction
    sum = 0
    for i in range(4):
        sum += m_j[i][0]*m_j[(i+1)%4][1]*m_j[(i+2)%4][1]*m_j[(i+3)%4][1]
    n_factor = product + sum

    # calcul m = [m({ABS}), m({HIPS}), m({PE}), m({PP}), m(THETA)]
    m = np.zeros(4)
    for i in range(4):
        m[i] = ( m_j[i][0]*m_j[(i+1)%4][1]*m_j[(i+2)%4][1]*m_j[(i+3)%4][1] ) / n_factor

    # m[4] = 0 # we force the classifier to choose one class
    return m

Then, the variables permit computing eknn

In [108]:
'''
    EKNN uses the pignistic probability and predicts the class.
    It returns the prediction and the real value
'''
def eknn(index, train_size, y, k):

    # Considering k neightbours
    index_voisins, _ , _ = voisins(index, train_size, y, k)
    
    # Compute pignistic probability (BetP)
    betp = np.zeros(4)  
    
    #add each mass associated at each neightbour and divide by the size (k)
    for j in range(len(index_voisins)):
        betp += m_j(index_voisins[j], train_size, y, k)
        
    #betp[4] /= 4 # |THETA|=4
    
    indice_classe_pred = np.argsort(betp)[0]
    
    classe_pred = classe[indice_classe_pred]
    
    return classe_pred, y[index]

In [109]:
pred, real = eknn(10029, len(X_train), y, 4)

In [110]:
pred, real

('PP', 'PP')

In [114]:
#Try the EKNN with 10 neightbours

true = 0
false = 0
k = 10
for i in range(len(X_train), len(big_df)):
    pred, expected = eknn(i, len(X_train), Y, k)
    if pred == expected :
        true += 1
    else :
        false += 1

In [115]:
print(true/ (len(big_df)-len(X_train)))

0.9205265724037055


The accuracy is around 93-95% with 10 neighbours

In [ ]:
X = [1, 2, 3, 4, 5, 10, 15, 20, 25, 35, 50]
accuracy= np.zeros(len(X))

for k in range(len(X)):
    true = 0
    false = 0
    k = X[k]
    for i in range(len(X_train), len(data)):
        pred, expected = eknn(i, len(X_train), y, k)
        if pred == expected :
            true += 1
        else :
            false += 1
    accuracy[k] = true / (len(data) - len(X_train))

Observing the evolution of accuracy with differents value of k

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Evolution of k")
plt.ylabel("Accuracy")
plt.title("Evolution of accuracy according to k")
plt.plot()

### c. Implement the expected utility model with eknn.

In [ ]:
def expected_utility_eknn(index, train_size, y, k, utility_matrix):
    # Considering k neighbors
    index_voisins, _, _ = voisins(index, train_size, y, k)

    # Compute pignistic probability (BetP)
    betp = np.zeros(4)

    # Add each mass associated with each neighbor and divide by the size (k)
    for j in range(len(index_voisins)):
        betp += m_j(index_voisins[j], train_size, y, k)
    betp /= len(index_voisins)

    # Use the utility matrix to compute expected utility
    expected_utilities = np.transpose(np.dot(utility_matrix.values, np.transpose(betp)))
    print(expected_utilities.shape)
    # Find the index of the class with the maximum expected utility
    indice_classe_pred = np.argmax(expected_utilities)
    
    classe_pred = classe[indice_classe_pred]

    return classe_pred, y[index]

In [ ]:
#Try the EKNN with 10 neightbours

true_utility = 0
bad_prediction = []
not_predict = []
k = 10

true_utility = 0
for i in range(len(X_train), len(data)):
    pred, expected = expected_utility_eknn(i, len(X_train), y, 10, utility_matrix)
    if pred == expected :
        true_utility += 1
    else :
        bad_prediction.append(pred)
        not_predict.append(expected)

(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)


In [ ]:
print(true_utility/ (len(data)-len(X_train)))

0.0009751340809361287
